In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# LangChain Expression Language (LCEL)

A Linguagem de Expressão LangChain, ou LCEL, é uma forma declarativa de **compor cadeias de maneira fácil**. A LCEL foi projetada desde o primeiro dia para suportar a colocação de protótipos em produção, sem a necessidade de alterações no código, desde a cadeia mais simples “prompt + LLM” até as cadeias mais complexas (já vimos pessoas executando com sucesso cadeias LCEL com centenas de etapas em produção). Para destacar alguns dos motivos pelos quais você pode querer usar a LCEL:
- **Suporte a streaming de primeira classe**: menor tempo possível para saída do primeiro token produzidio;
- **Suporte assíncrono**: Qualquer cadeia construída com a LCEL pode ser chamada tanto com a API síncrona;
- **Execução paralela otimizada**: Sempre que suas cadeias LCEL tiverem etapas que podem ser executadas em paralelo, automaticamente é feito isso;
- **Retentativas e fallbacks**: É maneira de tornar suas cadeias mais confiáveis em grande escala, na qual ações alternativas podem ser tomadas no caso de um erro em uma cadeia
- **Acessar resultados intermediários**: auxiliando na depuração de uma cadeia;

## Um exemplo simples de LCEL

O exemplo mais simples que podemos mostrar seria na construção de *prompt + model*

In [2]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI()

In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template('Crie uma frase sobre o assunto: {assunto}')

In [4]:
chain = prompt | chat

chain.invoke({'assunto': 'Arcane'})

AIMessage(content='"A série Arcane é uma obra-prima que mistura magia, ação e drama de forma surpreendente."', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 18, 'total_tokens': 44, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-261aa63d-2a77-48dd-8bb9-05aa8e2a7230-0', usage_metadata={'input_tokens': 18, 'output_tokens': 26, 'total_tokens': 44, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Adicinando mais elementos a chain

In [6]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | chat | StrOutputParser()
chain.invoke({'assunto': 'Computador Quantico'})

'"O computador quântico promete revolucionar a computação, resolvendo problemas complexos de forma exponencialmente mais rápida do que os computadores tradicionais."'

In [8]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template('Escreva uma breve descrição sobre o assunto: {assunto}')

chain = prompt | ChatOpenAI() | StrOutputParser()
chain.invoke({'assunto': 'Brasil'})

'O Brasil é o maior país da América do Sul, conhecido por sua diversidade cultural, paisagens deslumbrantes e povo acolhedor. Com uma extensão territorial vasta, o país abriga uma rica biodiversidade, incluindo a Amazônia, a maior floresta tropical do mundo. Além disso, o Brasil possui uma mistura única de influências indígenas, africanas, europeias e asiáticas, refletida em sua culinária, música, dança e festividades. Com uma economia diversificada e em crescimento, o país é uma potência regional e global, atraindo turistas e investidores de todo o mundo. Apesar dos desafios sociais e econômicos, o Brasil é um país vibrante e cheio de oportunidades.'

### ATENÇÃO! A ordem importa

In [ ]:
# chain = ChatOpenAI() | prompt | StrOutputParser()
# chain.invoke({'assunto': 'Brasil'})

ValueError: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.

### Como eu sei a ordem?

Para atingir o mesmo objetivo sem a criação da chain, os passos que eu deveria seguir seriam:
- Formatar o prompt template
- Enviar o prompt formatado para o modelo
- Fazer o parseamento do saída do modelo
Essa mesma ordem deve ser seguida para não termos erros

In [10]:
input = {'assunto': 'Brasil'}

In [11]:
prompt_formatado = prompt.invoke(input)
prompt_formatado

ChatPromptValue(messages=[HumanMessage(content='Escreva uma breve descrição sobre o assunto: Brasil', additional_kwargs={}, response_metadata={})])

In [12]:
resposta = chat.invoke(prompt_formatado)
resposta

AIMessage(content='O Brasil é o quinto maior país do mundo em extensão territorial e o sexto mais populoso, localizado na América do Sul. É conhecido por sua diversidade cultural, social e natural, com uma rica história que mistura influências indígenas, europeias, africanas e asiáticas. Possui uma economia variada, com destaque para a agricultura, indústria, serviços e recursos naturais. O país é famoso por suas belezas naturais, como a Floresta Amazônica, as praias paradisíacas, as Cataratas do Iguaçu e o Pantanal. A cultura brasileira é marcada pela música, dança, culinária e festividades populares, como o Carnaval. Apesar de seus desafios, o Brasil continua sendo um país de grandes potencialidades e oportunidades.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 190, 'prompt_tokens': 20, 'total_tokens': 210, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_toke

In [13]:
StrOutputParser().invoke(resposta)

'O Brasil é o quinto maior país do mundo em extensão territorial e o sexto mais populoso, localizado na América do Sul. É conhecido por sua diversidade cultural, social e natural, com uma rica história que mistura influências indígenas, europeias, africanas e asiáticas. Possui uma economia variada, com destaque para a agricultura, indústria, serviços e recursos naturais. O país é famoso por suas belezas naturais, como a Floresta Amazônica, as praias paradisíacas, as Cataratas do Iguaçu e o Pantanal. A cultura brasileira é marcada pela música, dança, culinária e festividades populares, como o Carnaval. Apesar de seus desafios, o Brasil continua sendo um país de grandes potencialidades e oportunidades.'

### Para não errar a ordem

É importante entendermos que cada componente recebe um tipo de entrada e gera um tipo de saída, e estes tipos precisam casar:

| Component       | Tipo de Entrada                                    | Tipo de Saída        |
|-----------------|-----------------------------------------------------|----------------------|
| Prompt          | Dicionário                                         | PromptValue      |
| ChatModel       | String única, lista de mensagens de chat ou PromptValue | Mensagem de Chat     |
| LLM             | String única, lista de mensagens de chat ou PromptValue | String               |
| OutputParser    | A saída de um LLM ou ChatModel                     | Depende do parser    |
| Retriever       | String única                                       | Lista de Documentos  |
| Tool            | String única ou dicionário, dependendo da ferramenta| Depende da ferramenta|

Podemos verificar isso pelos output e input schemas:

In [14]:
prompt.input_schema.model_json_schema()

{'properties': {'assunto': {'title': 'Assunto', 'type': 'string'}},
 'required': ['assunto'],
 'title': 'PromptInput',
 'type': 'object'}

In [15]:
prompt.output_schema.model_json_schema()

{'$defs': {'AIMessage': {'additionalProperties': True,
   'description': 'Message from an AI.\n\nAIMessage is returned from a chat model as a response to a prompt.\n\nThis message represents the output of the model and consists of both\nthe raw output as returned by the model together standardized fields\n(e.g., tool calls, usage metadata) added by the LangChain framework.',
   'properties': {'content': {'anyOf': [{'type': 'string'},
      {'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]},
       'type': 'array'}],
     'title': 'Content'},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'response_metadata': {'title': 'Response Metadata', 'type': 'object'},
    'type': {'const': 'ai',
     'default': 'ai',
     'enum': ['ai'],
     'title': 'Type',
     'type': 'string'},
    'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'title': 'Name'},
    'id': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'd

### Como eu teria feito com chains clássicas?

In [17]:
from langchain.chains.llm import LLMChain
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

chat = ChatOpenAI()
prompt = ChatPromptTemplate.from_template('Crie uma frase sobre o assunto: {assunto}')
output_parser = StrOutputParser()

In [18]:
chain = LLMChain(
    llm=chat,
    prompt= prompt,
    output_parser= output_parser
)

chain.invoke({'assunto': 'Séries'})

C:\Users\maris\AppData\Local\Temp\ipykernel_5160\3129580668.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


{'assunto': 'Séries',
 'text': '"Assistir séries é como embarcar em uma viagem emocionante para universos paralelos, onde cada episódio nos surpreende e nos faz querer mais."'}

## Desafio - Chains com LCEL

In [1]:
from dotenv import load_dotenv
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

# -----------------------------------------------------
# 1. Uma chain para pegar texto em outra língua para o português
# 2. Uma para resumir um texto
# 3. Uma chain que é a combinação da chain 1 com a chain 2
# -----------------------------------------------------

texto = """Video games have become one of the most popular forms of entertainment in the world. What started as simple pixelated experiences in the 1970s has evolved into immersive, realistic, and complex virtual worlds that captivate millions of players across the globe.
One of the most significant aspects of video games is their diversity. There are games for all kinds of interests—action, adventure, sports, puzzle, strategy, and simulation, just to name a few. This wide variety allows people of all ages and backgrounds to find something they enjoy.
Beyond entertainment, video games have also contributed to education, social connection, and even mental health. Educational games help students learn in interactive ways, while online multiplayer games allow people to connect and build friendships, regardless of distance. Some studies also suggest that video games can improve cognitive skills such as problem-solving, memory, and coordination.
However, video games are not without criticism. Concerns have been raised about addiction, excessive screen time, and exposure to violent content. It's important for players, especially younger ones, to find a healthy balance between gaming and other life activities.
Overall, video games are more than just a pastime—they are a cultural and technological phenomenon that continues to shape the way we learn, connect, and have fun in the digital age."""

# 1. Chain para traduzir par o Português
prompt_traducao = ChatPromptTemplate.from_messages(
    [
        ('system','Você é um assistente de tradução, traduza o texto para o idioma {idioma}.'),
        ('human', '{texto}')
    ]
)

chain_traducao = prompt_traducao | ChatOpenAI(model='gpt-4o-mini') | StrOutputParser()

# 2. Chain para resumir o texto traduzido
prompt_resumo = ChatPromptTemplate.from_messages(
    [
        ('system', 'Você é um assistente de resumos, aponte as partes mais importantes do texto em até 5 linhas'),
        ('human', '{texto_resumo}')
    ]
)

chain_resumo = prompt_resumo | ChatOpenAI(model='gpt-4o-mini') | StrOutputParser()

# 3. Chain que une as outras 2

chain_final = chain_traducao | chain_resumo
resultado = chain_final.invoke({'idioma': 'português', 'texto': texto})
print(resultado)


Os videogames se tornaram uma forma de entretenimento global, evoluindo de simples experiências para mundos virtuais complexos. Sua diversidade permite que pessoas de diferentes idades e interesses encontrem jogos que gostam. Além de entreter, contribuem para a educação, conexão social e saúde mental, melhorando habilidades cognitivas. No entanto, há críticas sobre vício e exposição a conteúdos violentos, destacando a necessidade de um equilíbrio saudável. Em suma, os videogames são um fenômeno cultural significativo na era digital.
